In [22]:
import json

with open('aa.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

with open('aa.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

In [60]:
with open('_image1661130828152_R.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

In [65]:
len(data['annotations'])

29

# ufo → coco

In [51]:
import json
with open('./_image1661130828152_R.json', 'r', encoding='utf-8') as f:
    dda = json.load(f)
a = dda['annotations'][7]['points']
arr = []
a[0]
for i in a:
    arr.extend(i)
arr

[802,
 828,
 807,
 813,
 814,
 806,
 825,
 794,
 835,
 783,
 848,
 777,
 866,
 775,
 878,
 779,
 886,
 786,
 889,
 793,
 895,
 807,
 896,
 815,
 898,
 821,
 901,
 833,
 903,
 843,
 898,
 846,
 897,
 854,
 893,
 861,
 891,
 875,
 887,
 909,
 886,
 949,
 887,
 978,
 886,
 1010,
 889,
 1062,
 895,
 1097,
 902,
 1123,
 909,
 1146,
 916,
 1157,
 920,
 1170,
 924,
 1176,
 924,
 1184,
 928,
 1191,
 932,
 1197,
 935,
 1206,
 936,
 1215,
 941,
 1222,
 942,
 1229,
 942,
 1238,
 936,
 1246,
 924,
 1250,
 912,
 1248,
 902,
 1240,
 888,
 1225,
 876,
 1219,
 863,
 1217,
 857,
 1220,
 852,
 1223,
 841,
 1222,
 835,
 1221,
 830,
 1216,
 828,
 1205,
 827,
 1195,
 828,
 1183,
 834,
 1172,
 839,
 1162,
 844,
 1140,
 840,
 1059,
 836,
 997,
 827,
 913,
 819,
 882,
 811,
 869,
 804,
 868,
 798,
 852]

In [69]:
from typing import Dict
import json
import datetime
import os

now = datetime.datetime.now()
now = now.strftime('%Y-%m-%d %H:%M:%S')

input_path = './_image1661130828152_R.json'
output_path = './coco_image1661130828152_R.json'

info = {
    'year': 2024,
    'version': '1.0',
    'description': 'OCR Competition Data',
    'contributor': 'Naver Boostcamp',
    'url': 'https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000273/data/data.tar.gz',
    'date_created': now
}
licenses = [{
    'id': '1',
    'name': 'For Naver Boostcamp Competition',
    'url': None
}]
categories = [{
    'id': 1,
    'name': 'word'
}]

def xywh(point):
    xy = []
    for i in point:
        xy.extend(i)
    segmentation = xy
    min_x = min(xy[0::2])
    min_y = min(xy[1::2])
    max_x = max(xy[0::2])
    max_y = max(xy[1::2])
    width = max_x - min_x
    height = max_y - min_y
    return [segmentation], min_x, min_y, width, height

def ufo_to_coco(data: Dict, output_path: str) -> None:
    img_id = 1
    annotation_id = 1
    annotations = []

    image = [{
        "id": 1,
        "width": data['metadata']['width'],
        "height": data['metadata']['height'],
        "file_name": data['filename'],
        "license": 0,
        "flickr_url": "",
        "coco_url": "",
        "date_captured": now
    }]

    # images.append(image)
    for annotation in data['annotations']:
        s_points, min_x, min_y, width, height = xywh(annotation['points'])

        coco_annotation = {
            "id": annotation_id,
            "image_id": img_id,
            "category_id": 1,
            "segmentation": s_points,
            "area": width * height,
            "bbox": [min_x, min_y, width, height],
            "iscrowd": 0,
            'attributes' : {}
        }
        annotations.append(coco_annotation)
        annotation_id += 1
        
    coco = {
        'licenses' : licenses,
        'info' : info,
        'categories' : categories,
        'images' : image,
        'annotations' : annotations
    }
    with open(output_path, 'w') as f:
        json.dump(coco, f, indent=4)

with open(input_path, 'r') as f:
    file = json.load(f)
ufo_to_coco(file, output_path)

print('-처리 완료')

-처리 완료


NameError: name 'coco' is not defined

# coco → ufo

In [38]:
from typing import Dict
import json
import datetime

now = datetime.datetime.now()
now = now.strftime('%Y-%m-%d %H:%M:%S')

input_path = './aa.json'   # coco 입력 json 파일명
output_path = './_aa.json' # ufo 출력 json 파일명

ufo = {
    'images': {}
}

def coco_seg_to_ufo(points):
    xy_list = [ [points[i], points[i+1]] for i in range(0, len(points), 2) ]
    return xy_list

def coco_to_ufo(file: Dict, output_path: str) -> None:
    anno_id = 1
    for annotation in file['annotations']:
        file_info = file['images'][int(annotation['image_id'])-1]
        image_name = file_info['file_name']
        if image_name not in ufo['images']:
            anno_id = 1
            ufo['images'][image_name] = {
                "paragraphs": {},
                "words": {},
                "chars": {},
                "img_w": file_info["width"],
                "img_h": file_info["height"],
                "tags": file_info, # 이거 변경
                "relations": {},
                "annotation_log": {
                    "worker": "",
                    "timestamp": now,
                    "tool_version": "LabelMe or CVAT",
                    "source": None
                    },
                "license_tag": {
                    "usability": True,
                    "public": False,
                    "commercial": True,
                    "type": None,
                    "holder": "Upstage"
                    }
                }
            
            # anno_id = 1
        ufo['images'][image_name]['words'][str(anno_id).zfill(4)] = {
            "transcription": "",
            "points":  coco_seg_to_ufo(annotation["segmentation"][0]),
            "orientation": "Horizontal",
            "language": None,
            "confidence": None,
            "illegibility": False
        }
        anno_id += 1

    with open(output_path, "w") as f:
        json.dump(ufo, f, indent=4)

with open(input_path, 'r') as f:
    file = json.load(f)
coco_to_ufo(file, output_path)

print('-처리 완료-')

-처리 완료-
